# Notebook 16: Expectation Maximization in practice


## Learning Goal 
The goal of this notebook is to gain intuition for Expectation Maximization using a simple example involving coin tosses.

## Overview

In Section XIV, we introduce Expectation-Maximization (EM) as a practical way to perform maximum likelihood estimation (MLE) even when some of the data is hidden (i.e in the presence of latent or hidden variables). To better understand EM, in this short notebook we'll explore a very simple coin-tossing example adapted from [Do and Batzoglou, Nat. Biotechnol. (2008)](https://www.nature.com/articles/nbt1406). 

Suppose that we are given two coins A and B with unkown bias $\theta_A$ and $\theta_B$, respectively. Our goal is to estimate the bias vector $\boldsymbol{\theta}= (\theta_A, \theta_B)$ from the outcomes of the following experiment: 

<blockquote> 
First choose one coin at random. Then toss the selected coin 10 times independently and record the number of heads observed. Repeat this procedure 5 times.
</blockquote>

Formally, let $z_i\in\{A,B\}$ be the coin selected in experiment $i$ and $x_i\in\{0,1,\cdots 10\}$ be the number heads recorded by tossing $z_i$ 10 times. Since we conduct $n=5$ such experiments, we can summarize the outcomes of these 50 tosses by two vectors: $\boldsymbol{x}=(x_1,x_2\cdots, x_5)$ and $\boldsymbol{z}=(z_1,z_2,\cdots, z_5)$.




### Exercise 1: What if we know everything?

 * Consider first the case where we have complete knowledge of the experiment, namely, both $\boldsymbol{x}$ and $\boldsymbol{z}$ are known. How would you intuitively estimate the biases of the two coins  $\boldsymbol{\theta}= (\theta_A, \theta_B)$ ?
 $\color{red}{\text{We could simply flip each coin N times and record the number of heads, NHead. Then the bias is NHead/N}}$
 
 * What's the likelihood of observing the complete outcomes of these experiments? In other words, what is $P(\boldsymbol{x},\boldsymbol{z}| n,\boldsymbol{\theta} )$? You may assume this is a Bernoulli trial. Namely, every time coin A(B) is tossed, we have, with probability $\theta_A$($\theta_B$), that the outcome is heads.
* What's the Maximum Likelihood Estimator (MLE)? Is this consistent with your intuition? 

Since we know everything, we can work with just one coin at a time. Let's say we are working with coin A and have observed k heads and n-k tails. The pobability of observing this result is 
 $$P(\theta_A) = C(n, k) * {\theta_A}^k * (1-\theta_A)^{(n-k)}.$$
 The loglikelihood is 
 $$\log(P(\theta_A)) = \log(C(n, k)) + k*\log(\theta_A) + (n-k)*\log(1-\theta_A),$$
 which is maximized when 
 $$\frac{d(\log(P(\theta_A)))}{d\theta_A} = (k/\theta_A) - (n-k)/(1-\theta_A) = 0.$$
 Solving for $\theta_A$ gives
 $$\theta_A = k/n,$$
 which is consistant with intuition.
 

## Comparing MLE and EM

To test your answer, let's do some numerics! We will compare the MLE estimates of biases with an Expectation Maximization procedure where we do not know ${\bf z}$. The following code computes our best guess for the biases using MLE -- assuming we know the identity of the coin used -- and compares it estimates arrived at using an EM procedure where we have no knowledge about which coin was being tossed (though we know the same coin was tossed 10 times).

In [3]:
import numpy as np
from scipy.special import comb
import math


def compute_likelihood(obs, n, pheads): # No surprise, it's Binomial!!!

    likelihood = comb(n, obs, exact=True)*(pheads**obs)*(1.0-pheads)**(n-obs)

    return likelihood

# generate experiments
num_coin_toss = 10 # each experiment contains num_coin_toss tosses
num_exp = 5  # we perform 5 such experiments
theta_A_true = 0.8 
theta_B_true = 0.4
coin_choice = np.zeros(num_exp) # initialize: 0 for A and 1 for B
head_counts = np.zeros(num_exp)

# MLE 
MLE_A = 0.0
MLE_B = 0.0

# generate the outcomes of experiment
for i in np.arange(num_exp):
    
    if np.random.randint(2) == 0: # coin A is selected
        head_counts[i] = np.random.binomial(num_coin_toss , theta_A_true, 1) # toss coin A num_coin_toss times
        MLE_A = MLE_A +  head_counts[i] # add the number of heads observed to total headcounts 
    
    else: # coin B is selected 
        head_counts[i] = np.random.binomial(num_coin_toss , theta_B_true, 1) # toss coin B num_coin_toss times
        coin_choice[i] = 1  # record the selection of coin B during experiment i 
        MLE_B = MLE_B +  head_counts[i] # add the number of heads observed to total headcounts 
    
tail_counts = num_coin_toss - head_counts


# MLE is merely the proportion of heads for each coin toss
MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice))*num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice)*num_coin_toss)



# initialize the pA(heads) and pB(heads), namely, coin biases
pA_heads = np.zeros(100); 
pB_heads = np.zeros(100); 

pA_heads[0] = 0.60 # initial guess
pB_heads[0] = 0.50 # initial guess

# E-M begins!
epsilon = 0.001   # error threshold
j = 0 # iteration counter
improvement = float('inf')

while (improvement > epsilon):
    
    expectation_A = np.zeros((num_exp,2), dtype=float) 
    expectation_B = np.zeros((num_exp,2), dtype=float)
    
    for i in np.arange(min(len(head_counts),len(tail_counts))):
        
        eH = head_counts[i]
        eT = tail_counts[i]
        
        # E step:
        lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
        lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])
        
        weightA = lA / (lA + lB)
        weightB = lB / (lA + lB)
              
        expectation_A[i] = weightA*np.array([eH, eT])
        expectation_B[i] = weightB*np.array([eH, eT])

  
    # M step
    theta_A = np.sum(expectation_A, axis = 0)[0] / np.sum(expectation_A) 
    theta_B = np.sum(expectation_B, axis = 0)[0] / np.sum(expectation_B) 

    print('At iteration %d, theta_A = %2f,  theta_B = %2f' % (j, theta_A, theta_B))
    
    pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A)); 
    pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B)); 

    improvement = max( abs(np.array([pA_heads[j+1],pB_heads[j+1]]) - np.array([pA_heads[j],pB_heads[j]]) ))
    j = j+1

# END of E-M, print the outcome

print('E-M converges at iteration %d' %j)
print('RESULT:')
print('E-M: theta_A = %2f,  theta_B = %2f' % (theta_A, theta_B))
print('MLE with complete data: theta_A = %2f,  theta_B = %2f' % (MLE_A, MLE_B))

At iteration 0, theta_A = 0.778697,  theta_B = 0.520572
At iteration 1, theta_A = 0.838668,  theta_B = 0.426590
At iteration 2, theta_A = 0.845417,  theta_B = 0.386892
At iteration 3, theta_A = 0.842258,  theta_B = 0.373503
At iteration 4, theta_A = 0.839269,  theta_B = 0.365950
At iteration 5, theta_A = 0.837132,  theta_B = 0.360681
At iteration 6, theta_A = 0.835595,  theta_B = 0.356802
At iteration 7, theta_A = 0.834462,  theta_B = 0.353874
At iteration 8, theta_A = 0.833608,  theta_B = 0.351626
At iteration 9, theta_A = 0.832954,  theta_B = 0.349880
At iteration 10, theta_A = 0.832447,  theta_B = 0.348510
At iteration 11, theta_A = 0.832050,  theta_B = 0.347428
At iteration 12, theta_A = 0.831737,  theta_B = 0.346568
E-M converges at iteration 13
RESULT:
E-M: theta_A = 0.831737,  theta_B = 0.346568
MLE with complete data: theta_A = 0.866667,  theta_B = 0.400000


### Exercise 2

 * How fast does EM converge? Is the converged result close to what you'd get from MLE? 
 The EM algorithm converges within about 10 steps: fairly quickly.
 * Following Exercise 1, what's the objective function we're optimizing in the E-step? Does this function have a *unique global maximum*? 
 The EM algorithm guarantees that the log-likelihood increases (or at least does not decrease) with each iteration, eventually converging to a local maximum. However, this local maximum may not be unique, and the algorithm does not guarantee that it will find a global maximum.
 * Compare both the results of MLE and EM to the actual bias (i.e. *theta_A_true*  and *theta_B_true* in the snippet above), comment on their performance.
 In some cases the EM and MLE are quite similar. However, in other cases EM seems to get stuck in local maxima.


## Final remarks: a few practical tricks

From Exercise 2 and Section XIV, we know that the E-M algorithm often approximates the MLE even in the presence of latent (hidden variables). Like with most optimization methods for nonconcave functions, E-M only guarantees convergence to a local maximum of the objective function. For this reason, its performance can be boosted by running the EM procedure starting with multiple initial parameters. 

### Exercise 3

* Now instead of having a fixed initial guess of coin biases (i.e. *pA_heads[0]* and *pB_heads[0]* in the snippet), draw these values uniformly at random from $[0,1]$ and run the E-M algorithm. Repeat this twenty times and report what you observed. What's the best initial guess that gives the closest estimate to the true parameters?

* As we discussed in Section X (LinReg), **Maximum a posteriori (MAP)** estimation differs from MLE in that it employs an augmented objective function which incorporates a prior distribution over the quantities we want to estimate, and the prior distribution can be think of as a regularizer for the objective fuction used in MLE. Here we will explore how to extend E-M to MAP estimation. 

  (1) First derive the MAP estimate for the one-coin-flipping example, namely,
  $$
  \hat{{\theta}}_{MAP}(\boldsymbol{x}) = \arg\max_{\theta\in[0,1]} \log P(\boldsymbol{x}|n,{\theta} ) + \log P({\theta}),
  $$
  where 
  $$P(\boldsymbol{x}|n,{\theta}) = \prod_{i=1}^{10} \text{Binomial} (x_i|n,\theta)$$
  
  $$P({\theta})=\mathcal{N}(\theta|\mu, \sigma)$$
  
  (2) Based on (1), now modify the E-M snippet above to incorporate this prior distribution into the **M-step**. Comment on the performance. For the prior choice, try $P(\boldsymbol{\theta})=\mathcal{N}(\theta_A|0.83, 1)\mathcal{N}(\theta_B|0.37, 1)$.

In [19]:
# Generate experiments
num_coin_toss = 10
num_exp = 5
theta_A_true = 0.8
theta_B_true = 0.4
coin_choice = np.zeros(num_exp)
head_counts = np.zeros(num_exp)

MLE_A = 0.0
MLE_B = 0.0

for i in np.arange(num_exp):
    if np.random.randint(2) == 0:
        head_counts[i] = np.random.binomial(num_coin_toss, theta_A_true, 1)
        MLE_A = MLE_A + head_counts[i]
    else:
        head_counts[i] = np.random.binomial(num_coin_toss, theta_B_true, 1)
        coin_choice[i] = 1
        MLE_B = MLE_B + head_counts[i]

tail_counts = num_coin_toss - head_counts

MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice)) * num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice) * num_coin_toss)


def em_algorithm(pA_init, pB_init):
    pA_heads = np.zeros(100)
    pB_heads = np.zeros(100)

    pA_heads[0] = pA_init
    pB_heads[0] = pB_init

    epsilon = 0.001
    j = 0
    improvement = float('inf')

    while (improvement > epsilon):

        expectation_A = np.zeros((num_exp, 2), dtype=float)
        expectation_B = np.zeros((num_exp, 2), dtype=float)

        for i in np.arange(min(len(head_counts), len(tail_counts))):
            eH = head_counts[i]
            eT = tail_counts[i]

            lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
            lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])

            weightA = lA / (lA + lB)
            weightB = lB / (lA + lB)

            expectation_A[i] = weightA * np.array([eH, eT])
            expectation_B[i] = weightB * np.array([eH, eT])

        theta_A = np.sum(expectation_A, axis=0)[0] / np.sum(expectation_A)
        theta_B = np.sum(expectation_B, axis=0)[0] / np.sum(expectation_B)

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A))
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B))

        improvement = max(abs(np.array([pA_heads[j+1], pB_heads[j+1]]) - np.array([pA_heads[j], pB_heads[j]])))
        j = j + 1

    return pA_heads[j], pB_heads[j]


# Run the EM algorithm 20 times with random initial guesses
num_runs = 20
results = []

for run in range(num_runs):
    pA_init = np.random.rand()
    pB_init = np.random.rand()
     
    theta_A_est, theta_B_est = em_algorithm(pA_init, pB_init)
    print("Initial guess:", pA_init, pB_init)
    print("Estimates:", theta_A_est, theta_B_est)
    results.append((pA_init, pB_init, theta_A_est, theta_B_est))
   
#
# Find the best initial guess that gives the closest estimate to the true parameters
min_error = float('inf')
best_initial_guess = None
best_estimates = None

for res in results:
    pA_init, pB_init, theta_A_est, theta_B_est = res
    error = np.abs(theta_A_true - theta_A_est) + np.abs(theta_B_true - theta_B_est)
    
    if error < min_error:
        min_error = error
        best_initial_guess = (pA_init, pB_init)
        best_estimates = (theta_A_est, theta_B_est)

print("Best initial guess:", best_initial_guess)
print("Best estimates:", best_estimates)

Initial guess: 0.13567752637182584 0.10304751602616857
Estimates: 0.8006244571475246 0.1345485977531261
Initial guess: 0.5800471754094352 0.699428670536462
Estimates: 0.13461312360610256 0.8006718975532251
Initial guess: 0.008510207292606764 0.749521227472404
Estimates: 0.13454852644317644 0.8006244083533406
Initial guess: 0.2375701953151188 0.04078991814210542
Estimates: 0.800615753783484 0.13453721326941898
Initial guess: 0.526705230848421 0.9267982411473378
Estimates: 0.25000017881718006 1.0
Initial guess: 0.8412259048142636 0.9914947858434703
Estimates: 0.25000017881758024 1.0
Initial guess: 0.11328785528138019 0.7020928427296269
Estimates: 0.13454476765349976 0.8006215660116202
Initial guess: 0.8523147326981977 0.1577837843320885
Estimates: 0.8006422742222696 0.13457270264619015
Initial guess: 0.42348415420493135 0.7098977659434903
Estimates: 0.13460722862809238 0.8006675954932113
Initial guess: 0.046762398622599566 0.9816199515575715
Estimates: 0.1346008539128457 0.80066293168004

Now, let's modify the E-M snippet to incorporate the prior distribution into the M-step. We will update the M-step to maximize the augmented objective function with the prior distribution. We use a Gaussian prior for both θ_A and θ_B.

In [21]:
import numpy as np
from scipy.special import comb
import math

def compute_likelihood(obs, n, pheads):
    likelihood = comb(n, obs, exact=True) * (pheads**obs) * (1.0-pheads)**(n-obs)
    return likelihood

def gaussian_prior(theta, mu, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((theta - mu) / sigma)**2)

# ... (previous code for generating experiments remains unchanged) ...

def em_algorithm_map(pA_init, pB_init):
    pA_heads = np.zeros(100)
    pB_heads = np.zeros(100)

    pA_heads[0] = pA_init
    pB_heads[0] = pB_init

    epsilon = 0.001
    j = 0
    improvement = float('inf')

    mu_A, sigma_A = 0.83, 1
    mu_B, sigma_B = 0.37, 1

    while (improvement > epsilon):

        # ... (previous code for E-step remains unchanged) ...

        # M step with prior
        theta_A = np.sum(expectation_A, axis=0)[0] / (np.sum(expectation_A) + (sigma_A**2) * (pA_heads[j] - mu_A))
        theta_B = np.sum(expectation_B, axis=0)[0] / (np.sum(expectation_B) + (sigma_B**2) * (pB_heads[j] - mu_B))

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A))
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B))

        improvement = max(abs(np.array([pA_heads[j+1], pB_heads[j+1]]) - np.array([pA_heads[j], pB_heads[j]])))
        j = j + 1

    return pA_heads[j], pB_heads[j]

# Run the EM algorithm 20 times with random initial guesses
num_runs = 20
results = []

for run in range(num_runs):
    pA_init = np.random.rand()
    pB_init = np.random.rand()

    theta_A_est, theta_B_est = em_algorithm_map(pA_init, pB_init)
    results.append((pA_init, pB_init, theta_A_est, theta_B_est))

# Report the results
print("Initial Guesses and MAP Estimates:")
print("Init θ_A, Init θ_B, MAP θ_A, MAP θ_B")
for result in results:
    print(f"{result[0]:.3f}, {result[1]:.3f}, {result[2]:.3f}, {result[3]:.3f}")

# Find the best initial guess that gives the closest estimate to the true parameters
min_diff = float('inf')
best_init_guess = None
best_estimates = None

for result in results:
    diff = abs(result[2] - theta_A_true) + abs(result[3] - theta_B_true)
    if diff < min_diff:
        min_diff = diff
        best_init_guess = (result[0], result[1])
        best_estimates = (result[2], result[3])

print("\nBest Initial Guess:")
print(f"θ_A: {best_init_guess[0]:.3f}, θ_B: {best_init_guess[1]:.3f}")

print("\nClosest Estimate to True Parameters:")
print(f"MAP θ_A: {best_estimates[0]:.3f}, MAP θ_B: {best_estimates[1]:.3f}")

print("\nTrue Parameters:")
print(f"θ_A: {theta_A_true}, θ_B: {theta_B_true}")


Initial Guesses and MAP Estimates:
Init θ_A, Init θ_B, MAP θ_A, MAP θ_B
0.780, 0.906, 0.832, 0.347
0.576, 0.067, 0.832, 0.347
0.460, 0.077, 0.832, 0.347
0.452, 0.276, 0.832, 0.347
0.063, 0.218, 0.832, 0.347
0.820, 0.942, 0.832, 0.347
0.827, 0.753, 0.832, 0.347
0.076, 0.803, 0.832, 0.347
0.686, 0.817, 0.832, 0.347
0.963, 0.619, 0.832, 0.347
0.980, 0.731, 0.832, 0.347
0.948, 0.277, 0.832, 0.347
0.621, 0.056, 0.832, 0.347
0.245, 0.824, 0.832, 0.347
0.674, 0.527, 0.832, 0.347
0.453, 0.146, 0.832, 0.347
0.232, 0.106, 0.832, 0.347
0.363, 0.880, 0.832, 0.347
0.516, 0.554, 0.832, 0.347
0.065, 0.327, 0.832, 0.347

Best Initial Guess:
θ_A: 0.780, θ_B: 0.906

Closest Estimate to True Parameters:
MAP θ_A: 0.832, MAP θ_B: 0.347

True Parameters:
θ_A: 0.8, θ_B: 0.4
